In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
import networkx as nx
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.nn import GAE, GCNConv
import networkx as nx
# 将边索引转换为 scipy 稀疏矩阵
import community as community_louvain  # Louvain community detection
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.utils import to_dense_adj, to_scipy_sparse_matrix, from_scipy_sparse_matrix
import scipy.sparse as sp
import numpy as np
import matplotlib.pyplot as plt
import torch
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community
from node2vec import Node2Vec
from sklearn.cluster import KMeans


class GraphAutoencoder(GAE):
    def __init__(self, encoder):
        super(GraphAutoencoder, self).__init__(encoder)

class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

#dataset = TUDataset(root='../tmp/Proteins', name='PROTEINS')
dataset = Planetoid(root='../tmp/Cora', name='Cora')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
print(data)
edge_index = data.edge_index.cpu().numpy()
num_nodes = data.num_nodes
G = nx.Graph()#创建networkx的空图
for i in range(num_nodes):
    G.add_node(i, features=data.x[i].cpu().numpy())
# 添加边
for i, j in edge_index.T:
    G.add_edge(i, j)
encoder = Encoder(in_channels=data.x.size(1), hidden_channels=16)
model = GraphAutoencoder(encoder)
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.train()

for epoch in range(200):
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    adj = torch.matmul(z, z.t())  # Approximate adjacency matrix
    adj_pos = torch.zeros_like(adj)
    adj_pos[data.edge_index[0], data.edge_index[1]] = 1

    # Compute the loss (Binary Cross Entropy Loss for link prediction)
    loss = nn.BCEWithLogitsLoss()(adj.view(-1), adj_pos.view(-1))
    
    loss.backward()
    optimizer.step()
    if epoch%10==0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Epoch 0: Loss = 0.7074
Epoch 10: Loss = 0.6958
Epoch 20: Loss = 0.6936
Epoch 30: Loss = 0.6935
Epoch 40: Loss = 0.6932
Epoch 50: Loss = 0.6932
Epoch 60: Loss = 0.6932
Epoch 70: Loss = 0.6932
Epoch 80: Loss = 0.6931
Epoch 90: Loss = 0.6931
Epoch 100: Loss = 0.6931
Epoch 110: Loss = 0.6931
Epoch 120: Loss = 0.6931
Epoch 130: Loss = 0.6931
Epoch 140: Loss = 0.6931
Epoch 150: Loss = 0.6931
Epoch 160: Loss = 0.6931
Epoch 170: Loss = 0.6931
Epoch 180: Loss = 0.6931
Epoch 190: Loss = 0.6931


In [10]:
from sklearn.cluster import KMeans
model.eval()
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)
num_clusters = 7  # You can choose a different number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(z.detach().numpy())

# Create node to community mapping
node_cluster_map = {node: clusters[i] for i, node in enumerate(G.nodes())}

# Convert node to community mapping to a list of communities
communities_kmeans = []
for community_id in set(node_cluster_map.values()):
    communities_kmeans.append([node for node in node_cluster_map if node_cluster_map[node] == community_id])

# Compute modularity
modularity_kmeans = community.modularity(G, communities_kmeans)
print(f"Graph Autoencoder + KMeans clustering modularity: {modularity_kmeans:.4f}")

Graph Autoencoder + KMeans clustering modularity: 0.0002


/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:1008: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
